In [1]:
#from fast.ai -> fastbook -> 09_tabular
#and  Practical-Deep-Learning-for-Coders-2.0 -> Tabular Notebooks -> 02_Regression_and_Permutation_Importance
#kaggle Rossmann competition for data
#Cleaned data source: fastai -> course-v3 -> nbs -> dl1 -> rossmann_data_clean.ipynb
#hide
from utils import *
# from kaggle import api
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai2.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

In [2]:
path = Path('data/MiniRossmann/combined/')
# path = Path('data/MiniRossmann/')


In [3]:
#hide
Path.BASE_PATH = path

In [4]:
train_df = pd.read_csv(path/'train_clean_multi1&3edited.csv', low_memory=False)
test_df = pd.read_csv(path/'test_clean_multi1&3edited.csv', low_memory=False)
# train_df = pd.read_csv(path/'train_clean1and3UNI.csv', low_memory=False)
# test_df = pd.read_csv(path/'test_clean1and3UNI.csv', low_memory=False)

# train_df = pd.read_csv(path/'train_clean.csv', low_memory=False)
# test_df = pd.read_csv(path/'test_clean.csv', low_memory=False)

In [5]:
train_df.head().T

,0,1,2,3,4
Date,1/2/2013,1/3/2013,1/4/2013,1/5/2013,1/7/2013
Unnamed: 0,843210,842101,840993,839886,838761
index,1014980,1013865,1012750,1011635,1009405
Store,1,1,1,1,1
DayOfWeek,3,4,5,6,1
...,...,...,...,...,...
3StateHoliday_bw,1,1,1,1,1
3Promo_bw,0,0,0,0,1
3SchoolHoliday_fw,3,2,1,0,0
3StateHoliday_fw,0,0,0,0,0


In [6]:
train_df.columns

Index(['Date', 'Unnamed: 0', 'index', 'Store', 'DayOfWeek', 'Sales',
       'Customers', 'Open', 'Promo', 'StateHoliday',
       ...
       '3AfterStateHoliday', '3BeforeStateHoliday', '3AfterPromo',
       '3BeforePromo', '3SchoolHoliday_bw', '3StateHoliday_bw', '3Promo_bw',
       '3SchoolHoliday_fw', '3StateHoliday_fw', '3Promo_fw'],
      dtype='object', length=187)

In [7]:
# cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen',
#     'Promo2Weeks', 'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear',
#     'State', 'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw',
#     'SchoolHoliday_fw', 'SchoolHoliday_bw', 'Promo', 'SchoolHoliday']

# cont_vars = ['CompetitionDistance', 'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
#    'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
#    'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
#    'AfterStateHoliday', 'BeforeStateHoliday']
   
# dep_var = 'Sales'

In [8]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen',
    'Promo2Weeks', 'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear',
    'State', 'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw',
    'SchoolHoliday_fw', 'SchoolHoliday_bw', 'Promo', 'SchoolHoliday', '3Store', '3DayOfWeek', '3Year', '3Month', '3Day', '3StateHoliday', '3CompetitionMonthsOpen',
    '3Promo2Weeks', '3StoreType', '3Assortment', '3PromoInterval', '3CompetitionOpenSinceYear', '3Promo2SinceYear',
    '3State', '3Week', '3Events', '3Promo_fw', '3Promo_bw', '3StateHoliday_fw', '3StateHoliday_bw',
    '3SchoolHoliday_fw', '3SchoolHoliday_bw', '3Promo', '3SchoolHoliday']

cont_vars = ['CompetitionDistance', 'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday', 'BeforeStateHoliday', '3CompetitionDistance', '3Max_TemperatureC', '3Mean_TemperatureC', '3Min_TemperatureC',
   '3Max_Humidity', '3Mean_Humidity', '3Min_Humidity', '3Max_Wind_SpeedKm_h', 
   '3Mean_Wind_SpeedKm_h', '3CloudCover', '3trend', '3trend_DE',
   '3AfterStateHoliday', '3BeforeStateHoliday']

dep_vars = ['Sales', '3Sales']

In [9]:
train_df[dep_vars] = np.log(train_df[dep_vars])


In [10]:
####setting up splits

In [11]:
procs = [FillMissing, Normalize, Categorify]
# procs = [Categorify, FillMissing]

In [12]:
len(train_df), len(test_df)

(779, 48)

In [13]:
test_df['Date'].min(), test_df['Date'].max()

('8/1/2015', '9/9/2015')

In [14]:
idx = train_df['Date'][(train_df['Date'] == train_df['Date'][len(test_df)])].index.max()

In [15]:
idx

48

In [16]:
splits = (L(range(idx, len(train_df))),L(range(idx)))

In [17]:
splits

((#731) [48,49,50,51,52,53,54,55,56,57...], (#48) [0,1,2,3,4,5,6,7,8,9...])

In [18]:
pd.options.mode.chained_assignment=None

In [19]:
to = TabularPandas(train_df, procs, cat_vars, cont_vars, y_names=dep_vars, splits=splits)

In [20]:
len(to.train),len(to.valid)

(731, 48)

We can see that the data is still displayed as strings for categories (we only show a few columns here because the full table is too big to fit on a page):

In [21]:
#hide_output
to.show(3)

,Store,DayOfWeek,Year,Month,Day,StateHoliday,CompetitionMonthsOpen,Promo2Weeks,StoreType,Assortment,PromoInterval,CompetitionOpenSinceYear,Promo2SinceYear,State,Week,Events,Promo_fw,Promo_bw,StateHoliday_fw,StateHoliday_bw,SchoolHoliday_fw,SchoolHoliday_bw,Promo,SchoolHoliday,3Store,3DayOfWeek,3Year,3Month,3Day,3StateHoliday,3CompetitionMonthsOpen,3Promo2Weeks,3StoreType,3Assortment,3PromoInterval,3CompetitionOpenSinceYear,3Promo2SinceYear,3State,3Week,3Events,3Promo_fw,3Promo_bw,3StateHoliday_fw,3StateHoliday_bw,3SchoolHoliday_fw,3SchoolHoliday_bw,3Promo,3SchoolHoliday,CloudCover_na,3CloudCover_na,CompetitionDistance,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,CloudCover,trend,trend_DE,AfterStateHoliday,BeforeStateHoliday,3CompetitionDistance,3Max_TemperatureC,3Mean_TemperatureC,3Min_TemperatureC,3Max_Humidity,3Mean_Humidity,3Min_Humidity,3Max_Wind_SpeedKm_h,3Mean_Wind_SpeedKm_h,3CloudCover,3trend,3trend_DE,3AfterStateHoliday,3BeforeStateHoliday,Sales,3Sales
48,1,3,2013,2,27,False,24,0,c,a,#na#,2008,1900,HE,9,Rain-Snow,2,2,0,0,0,0,0,0,3,3,2013,2,27,False,24,25,a,a,"Jan,Apr,Jul,Oct",2006,2011,NW,9,Rain,2,2,0,0,0,0,0,0,False,False,1270.0,2.0,1.0,1.0,100.0,82.0,70.0,21.0,16.0,8.0,59.0,55.0,57.0,-30.0,14130.0,1.0,1.0,-4.768372e-07,96.0,87.0,80.0,23.0,16.0,8.0,56.0,55.0,57.0,-30.0,8.424639,8.551015
49,1,4,2013,2,28,False,24,0,c,a,#na#,2008,1900,HE,9,Snow,3,1,0,0,0,0,0,0,3,4,2013,2,28,False,24,25,a,a,"Jan,Apr,Jul,Oct",2006,2011,NW,9,#na#,3,1,0,0,0,0,0,0,False,False,1270.0,5.0,3.0,1.0,87.0,73.0,58.0,16.0,10.0,8.0,59.0,55.0,58.0,-29.0,14130.0,6.0,3.0,1.000000e+00,87.0,77.0,59.0,14.0,10.0,7.0,56.0,55.0,58.0,-29.0,8.450198,8.726968
50,1,5,2013,3,1,False,24,0,c,a,#na#,2008,1900,HE,9,Rain,4,0,0,0,0,0,0,0,3,5,2013,3,1,False,24,25,a,a,"Jan,Apr,Jul,Oct",2006,2011,NW,9,Rain,4,0,0,0,0,0,0,0,False,False,1270.0,4.0,3.0,2.0,81.0,71.0,55.0,23.0,10.0,8.0,59.0,55.0,59.0,-28.0,14130.0,4.0,2.0,1.000000e+00,100.0,78.0,58.0,14.0,8.0,7.0,56.0,55.0,59.0,-28.0,8.436200,8.791030


In [22]:
#hide_output
to.items.head(3)

,Date,Unnamed: 0,index,Store,...,3StateHoliday_fw,3Promo_fw,CloudCover_na,3CloudCover_na
48,2/27/2013,789903,952540,1,...,1,3,1,1
49,2/28/2013,788791,951425,1,...,1,4,1,1
50,3/1/2013,787679,950310,1,...,1,5,1,1


In [23]:
(path/'to.pkl').save(to)

In [24]:
###decision tree

In [25]:
#hide
to = (path/'to.pkl').load()

In [26]:
xs,ys = to.train.xs,to.train.ys
valid_xs,valid_ys = to.valid.xs,to.valid.ys

In [37]:
m = DecisionTreeRegressor(max_leaf_nodes=4)
m.fit(xs, ys);

In [38]:
m = DecisionTreeRegressor()
m.fit(xs, ys);

In [39]:
def r_mse(pred,ys): return round(math.sqrt(((pred-ys)**2).mean()), 6)
def m_rmse(m, xs, ys): return r_mse(m.predict(xs), ys)

In [40]:
m_rmse(m, xs, ys)

TypeError: cannot convert the series to <class 'float'>

In [31]:
m_rmse(m, valid_xs, valid_ys)

TypeError: cannot convert the series to <class 'float'>

In [32]:
m.get_n_leaves(), len(xs)

(731, 731)

In [33]:
m = DecisionTreeRegressor(min_samples_leaf=25)
m.fit(to.train.xs, to.train.ys)
m_rmse(m, xs, ys), m_rmse(m, valid_xs, valid_ys)

TypeError: cannot convert the series to <class 'float'>

In [34]:
m.get_n_leaves()

23